# Food in Art

In [ ]:
import pandas as pd
import yaml


In [ ]:
# Load configuration from YAML file
with open("config.yaml", "r") as file:
    config = yaml.safe_load(file)
config

In [ ]:
data = pd.read_csv('data/paintings_with_food_mentions.csv')
data

In [ ]:
selected_columns = [list(col.keys())[0] for col in config['columns']]
print(selected_columns)
df = data[selected_columns]
df

In [ ]:
food_df = df[df['contains_food'] == 1]
food_df